# Regression

By: Oscar Ko

This notebook was created for data analysis and classification on this dataset from Stanford:

https://data.stanford.edu/hcmst2017

---
---

# Step 1: Imports and Data

In [3]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv("data/df_renamed.csv")

print(df.shape, "\n")

df.info(verbose=True)

(2844, 75) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2844 entries, 0 to 2843
Data columns (total 75 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        2844 non-null   int64  
 1   ID                                2844 non-null   int64  
 2   ageGap                            2844 non-null   float64
 3   attendReligiousServiceFreq        2844 non-null   object 
 4   employmentStatus                  2844 non-null   object 
 5   genderSubjectAttractedTo          2837 non-null   object 
 6   houseType                         2844 non-null   object 
 7   householdAdults_num               2844 non-null   int64  
 8   householdIncome                   2844 non-null   int64  
 9   householdMinor_num                2844 non-null   int64  
 10  householdSize                     2844 non-null   int64  
 11  interracial                       2822 non-null   object

### Test-Train Stratified Split

- Stratified split to ensure equal proportions of all labels in both sets.
- Use on "relationshipQuality_isGood"

First drop the other two outcome labels, leaving just "relationshipQuality_num"

In [4]:
# Remove "relationshipQuality" and "relationshipQuality_isGood"

df_copy = df.copy().drop(["relationshipQuality", "relationshipQuality_isGood"], axis=1)

success1 = "relationshipQuality_isGood" not in df_copy.columns
success2 = "relationshipQuality" not in df_copy.columns

print("Sucessfully removed both columns?", success1 & success2)

Sucessfully removed both columns? True


**The Split**

Since the "relationshipQuality_num" is a numeric variable with only 5 values, it can be stratified along the five values as if it were categorical, which helps create equally proportionate test and train sets. (The modeling will still treat it as a numeric variable.)

In [5]:
# import package
from sklearn.model_selection import train_test_split

# declare our X inputs and y outcomes
X = df_copy.drop("relationshipQuality_num", axis=1)
y = df_copy["relationshipQuality_num"]


# stratify split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state=42)

print("X_train.shape = ", X_train.shape)
print("X_test.shape = ", X_test.shape)

print("y_train.shape = ", y_train.shape)
print("y_test.shape = ", y_test.shape)

print("\n")
print("y_train class proportions: \n", y_train.value_counts(normalize=True))

print("\n")
print("y_test class proportions: \n", y_test.value_counts(normalize=True))

X_train.shape =  (2275, 72)
X_test.shape =  (569, 72)
y_train.shape =  (2275,)
y_test.shape =  (569,)


y_train class proportions: 
 5    0.599560
4    0.310330
3    0.071209
2    0.010989
1    0.007912
Name: relationshipQuality_num, dtype: float64


y_test class proportions: 
 5    0.599297
4    0.311072
3    0.070299
2    0.010545
1    0.008787
Name: relationshipQuality_num, dtype: float64


### Remove unneeded columns and missing values

The code is the same as in the classification notebook.

In [6]:

# removing columns --------------------------------------

columns_to_remove = [
    "moveIn_YearFraction",
    "shipStart_to_moveIn_YearFraction",
    "Unnamed: 0", # Also Remove column with no useful info
    "ID" # Also Remove column with no useful info
]

# anything done to the training set has to be done to the testing set
X_train.drop(columns_to_remove, axis=1, inplace=True)
X_test.drop(columns_to_remove, axis=1, inplace=True)


# Dropping NA values ------------------------------------

def cleanDatasets(X_data, y_data):
    
    cols_with_na = X_data.columns[X_data.isna().any()].tolist()
    
    for col in cols_with_na:
        
        indexes = X_data[col].notna()
        X_data = X_data[indexes]
        y_data = y_data[indexes]
    
    # filter odd partner age cases
    age_filter = X_data["partnerAge"] > 5
    X_data = X_data[age_filter]
    y_data = y_data[age_filter]
    
    return X_data, pd.DataFrame(y_data)

# anything done to the training set has to be done to the testing set        
X_train, y_train = cleanDatasets(X_train, y_train)
X_test, y_test = cleanDatasets(X_test, y_test)


# recombine training sets
training_set = pd.concat([X_train, y_train], axis=1)


# Check all cases and columns
print(training_set.shape)


print(X_train.shape, X_test.shape)

(2090, 69)
(2090, 68) (518, 68)


---
---

# Step 2: Prepare the Data for Machine Learning

The code is the same as in the classification notebook.

In [7]:
# First get the numeric columns and the categorical columns

numeric_features = X_train._get_numeric_data().columns

categorical_features = list(set(X_train.columns) - set(numeric_features))

print(len(X_train.columns), len(numeric_features), len(categorical_features))

print(X_train.shape, X_test.shape)

68 21 47
(2090, 68) (518, 68)


In [8]:
# One hot encode categorical features for the X_train and X_test sets

X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)

print(X_train.shape, X_test.shape)

(2090, 107) (518, 105)


#### Taking care of the extra columns

In [10]:
train_extra_cols = []

test_extra_cols = []


for col in X_test.columns:

    if col not in X_train.columns:
        
        test_extra_cols.append(col)
        
        
for col in X_train.columns:

    if col not in X_test.columns:
        
        train_extra_cols.append(col)
        
print(len(train_extra_cols), " extra train columns:\n",  train_extra_cols)
print(len(test_extra_cols), " extra test columns:\n",  test_extra_cols)

4  extra train columns:
 ['attendReligiousServiceFreq_Refused', 'subjectGrewUpInUS_Refused', 'partnerGender_[Partner Name] is Other, please specify', 'metOnline_nonDatingSite_yes']
2  extra test columns:
 ['isLivingTogether_Refused', 'subjectCountryWhenMetPartner_Refused']


**NOTES:** Some options within the categorical columns were not selected by any of the subjects within the **test** group, so those options did not become their own columns when one-hot encoding.

Some options within the categorical columns were not selected by any of the subjects within the **train** group, so those options did not become their own columns when one-hot encoding.

- I will create columns with just zeros to fill in this gap, so the machine learning models can work without error.

In [11]:
def addZeroCols(row, extra_cols):
    
    for col in extra_cols:
        
        row[col] = 0
        
    return row


X_test = X_test.apply(addZeroCols, args=[train_extra_cols], axis=1)
X_train = X_train.apply(addZeroCols, args=[test_extra_cols], axis=1)

print(X_train.shape, X_test.shape)

(2090, 109) (518, 109)


### Feature Scaling on Numeric Columns with Standardization

In [13]:
# import scaling & column transformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Create a function to scale the train and test sets

def scaleCategoricalFeatures(X_data):
    
    scaler = StandardScaler()

    X_data[numeric_features] = scaler.fit_transform(X_data[numeric_features])
    
    return X_data

X_train = scaleCategoricalFeatures(X_train)
X_test = scaleCategoricalFeatures(X_test)

print(X_train.shape, X_test.shape)

(2090, 109) (518, 109)


### Make sure it worked by seeing if Standard Deviations of Numeric Columns are 1

In [14]:
X_train.describe()

,ageGap,householdAdults_num,householdIncome,householdMinor_num,householdSize,isHispanic,met_YearFraction,met_to_shipStart_diff,numRelativesSeePerMonth,partnerAge,...,subjectRace_Asian or Pacific Islander,subjectRace_Black or African American,subjectRace_Other (please specify),subjectRace_White,houseType_A mobile home,houseType_A one-family house attached to one or more houses,houseType_A one-family house detached from any other house,"houseType_Boat, RV, van, etc.",isLivingTogether_Refused,subjectCountryWhenMetPartner_Refused
count,2.090000e+03,2.090000e+03,2.090000e+03,2.090000e+03,2.090000e+03,2.090000e+03,2.090000e+03,2.090000e+03,2.090000e+03,2.090000e+03,...,2090.000000,2090.000000,2090.00000,2090.000000,2090.000000,2090.000000,2090.000000,2090.000000,2090.0,2090.0
mean,6.209812e-17,1.375295e-16,-5.397065e-17,2.996009e-17,-2.291628e-16,2.267192e-16,9.687918e-15,9.298782e-17,2.287909e-16,-1.130409e-16,...,0.038756,0.087081,0.02201,0.814354,0.036842,0.082775,0.723445,0.000957,0.0,0.0
std,1.000239e+00,1.000239e+00,1.000239e+00,1.000239e+00,1.000239e+00,1.000239e+00,1.000239e+00,1.000239e+00,1.000239e+00,1.000239e+00,...,0.193059,0.282022,0.14675,0.388914,0.188419,0.275608,0.447402,0.030927,0.0,0.0
min,-8.615692e-01,-1.391770e+00,-1.349058e+00,-5.497378e-01,-1.262110e+00,-3.652474e-01,-3.308361e+00,-3.705394e-01,-6.710181e-01,-2.410680e+00,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,-6.535665e-01,-2.188671e-01,-7.142473e-01,-5.497378e-01,-5.376360e-01,-3.652474e-01,-7.691073e-01,-3.705394e-01,-6.710181e-01,-8.618190e-01,...,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,-2.375610e-01,-2.188671e-01,-1.914617e-01,-5.497378e-01,-5.376360e-01,-3.652474e-01,2.017838e-01,-3.308403e-01,-2.779059e-01,6.749774e-02,...,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0
75%,1.784444e-01,-2.188671e-01,2.939820e-01,4.398849e-01,7.301935e-01,-3.652474e-01,8.440656e-01,-1.126409e-01,3.117625e-01,7.489967e-01,...,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0
max,8.498554e+00,6.818551e+00,3.094619e+00,7.367244e+00,5.258156e+00,2.737871e+00,1.371824e+00,1.266423e+01,9.156788e+00,2.855448e+00,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0


In [15]:
# alphbetize column names

X_train = X_train.sort_index(axis=1)
X_test = X_test.sort_index(axis=1)

X_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2090 entries, 1087 to 2731
Data columns (total 109 columns):
 #    Column                                                                                        Dtype  
---   ------                                                                                        -----  
 0    ageGap                                                                                        float64
 1    attendReligiousServiceFreq_More than once a week                                              int64  
 2    attendReligiousServiceFreq_Never                                                              int64  
 3    attendReligiousServiceFreq_Once a week                                                        int64  
 4    attendReligiousServiceFreq_Once a year or less                                                int64  
 5    attendReligiousServiceFreq_Once or twice a month                                              int64  
 6    attendReligiousServ

---
---
# Step 3: Regression Modelling

### Full Model - Linear Regression

In [17]:
# select predictor variables and outcome variable for the model ---

X = X_train

y = y_train


# # To create and test interaction terms 

# X['Interaction_term'] = data['predictor_var'] * data['predictor_var2']


# build model ---------------------------------------

from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

# require a constant in the model summary
X = sm.add_constant(X)

# build the model and fit it to the data
linear_model = sm.OLS(y, X, hasconst=True)
linear_model = linear_model.fit()


linear_model.summary()

# NOTE: P-values are the probability the results are insignificant
# and due to random chance. 
# With Values lower than the alpha level (0.05), we can reject the null hypothesis

# NOTE: R-squared is the amount of variability within the outcome variable
# that is explained by our model

# NOTE: Interpreting coefficients
# If we hold all other features fixed, a one unit increase 
# in the predictor_variable is correlated with 
# an increase of (coefficient_number) for the outcome_variable

# Prob (F-statistic) = P-value of the model

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     relationshipQuality_num   R-squared:                       0.193
Model:                                 OLS   Adj. R-squared:                  0.150
Method:                      Least Squares   F-statistic:                     4.466
Date:                     Tue, 27 Sep 2022   Prob (F-statistic):           1.72e-41
Time:                             14:25:48   Log-Likelihood:                -2112.5
No. Observations:                     2090   AIC:                             4439.
Df Residuals:                         1983   BIC:                             5043.
Df Model:                              106                                         
Covariance Type:                 nonrobust                                         
================================================================================================================================================================
                                                                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------------
const                                                                                            4.5686      0.836      5.465      0.000       2.929       6.208
ageGap                                                                                          -0.0104      0.018     -0.591      0.554      -0.045       0.024
attendReligiousServiceFreq_More than once a week                                                 0.1268      0.069      1.834      0.067      -0.009       0.262
attendReligiousServiceFreq_Never                                                                -0.0375      0.049     -0.767      0.443      -0.133       0.058
attendReligiousServiceFreq_Once a week                                                           0.0306      0.053      0.579      0.563      -0.073       0.134
attendReligiousServiceFreq_Once a year or less                                                   0.0300      0.052      0.578      0.563      -0.072       0.132
attendReligiousServiceFreq_Once or twice a month                                                 0.0015      0.064      0.023      0.981      -0.124       0.127
attendReligiousServiceFreq_Refused                                                              -0.5539      0.352     -1.572      0.116      -1.245       0.137
employmentStatus_retired                                                                         0.0923      0.053      1.752      0.080      -0.011       0.196
employmentStatus_unemployed                                                                     -0.0415      0.041     -1.020      0.308      -0.121       0.038
genderSubjectAttractedTo_sexually attracted mostly to same gender, sometimes opposite gender     0.0781      0.163      0.479      0.632      -0.241       0.398
genderSubjectAttractedTo_sexually attracted only to opposite gender                              0.0849      0.090      0.946      0.344      -0.091       0.261
genderSubjectAttractedTo_sexually attracted only to same gender                                  0.2955      0.174      1.702      0.089      -0.045       0.636
genderSubjectAttractedTo_sexually attracted to men and women equally                             0.0452      0.106      0.427      0.669      -0.162       0.253
houseType_A mobile home                                                                          0.0694      0.097      0.714      0.475      -0.121       0.260
houseType_A one-family house attached to one or more houses                                     -0.0391      0.068     -0.570     

### Backwards Elimination (Round 1) - Linear Regression (alpha level = 0.05)

In [24]:
# select predictor variables and outcome variable for the model ---

features = [
    "householdIncome",
    "householdMinor_num",
    "isLivingTogether_Yes",
    "metAs_workNeighbors_yes",
    "metIn_school_yes",
    "partnerRace_Asian or Pacific Islander",
    "partnerRace_White",
    "sexFrequency_3 to 6 times a week",
    "sexFrequency_Once a month or less",
    "sexFrequency_Once or twice a week",
    "subjectRace_Asian or Pacific Islander",
    "whoEarnedMore_We earned about the same amount"
]

X = X_train[features]

y = y_train

# require a constant in the model summary
X = sm.add_constant(X)

# build the model and fit it to the data
linear_model = sm.OLS(y, X, hasconst=True)
linear_model = linear_model.fit()


linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     relationshipQuality_num   R-squared:                       0.136
Model:                                 OLS   Adj. R-squared:                  0.131
Method:                      Least Squares   F-statistic:                     27.26
Date:                     Tue, 27 Sep 2022   Prob (F-statistic):           5.43e-58
Time:                             14:51:02   Log-Likelihood:                -2183.4
No. Observations:                     2090   AIC:                             4393.
Df Residuals:                         2077   BIC:                             4466.
Df Model:                               12                                         
Covariance Type:                 nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                             3.9155      0.056     69.742      0.000       3.805       4.026
householdIncome                                   0.0673      0.016      4.333      0.000       0.037       0.098
householdMinor_num                               -0.0734      0.015     -4.764      0.000      -0.104      -0.043
isLivingTogether_Yes                              0.4575      0.043     10.682      0.000       0.373       0.541
metAs_workNeighbors_yes                           0.2984      0.130      2.304      0.021       0.044       0.552
metIn_school_yes                                  0.1580      0.049      3.199      0.001       0.061       0.255
partnerRace_Asian or Pacific Islander             0.2375      0.087      2.736      0.006       0.067       0.408
partnerRace_White                                 0.2127      0.044      4.889      0.000       0.127       0.298
sexFrequency_3 to 6 times a week                  0.2089      0.052      4.021      0.000       0.107       0.311
sexFrequency_Once a month or less                -0.2101      0.040     -5.299      0.000      -0.288      -0.132
sexFrequency_Once or twice a week                 0.1004      0.042      2.400      0.016       0.018       0.183
subjectRace_Asian or Pacific Islander            -0.1497      0.087     -1.728      0.084      -0.320       0.020
whoEarnedMore_We earned about the same amount     0.1041      0.046      2.260      0.024       0.014       0.194
==============================================================================
Omnibus:                      585.284   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1597.753
Skew:                          -1.467   Prob(JB):                         0.00
Kurtosis:                       6.121   Cond. No.                         14.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Backwards Elimination (Round 2) - Linear Regression (alpha level = 0.05)

In [26]:
# select predictor variables and outcome variable for the model ---

features = [
    "householdIncome",
    "householdMinor_num",
    "isLivingTogether_Yes",
    "metAs_workNeighbors_yes",
    "metIn_school_yes",
    "partnerRace_Asian or Pacific Islander",
    "partnerRace_White",
    "sexFrequency_3 to 6 times a week",
    "sexFrequency_Once a month or less",
    "sexFrequency_Once or twice a week",
    "whoEarnedMore_We earned about the same amount"
]

X = X_train[features]

y = y_train

# require a constant in the model summary
X = sm.add_constant(X)

# build the model and fit it to the data
linear_model = sm.OLS(y, X, hasconst=True)
linear_model = linear_model.fit()


linear_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     relationshipQuality_num   R-squared:                       0.135
Model:                                 OLS   Adj. R-squared:                  0.130
Method:                      Least Squares   F-statistic:                     29.43
Date:                     Tue, 27 Sep 2022   Prob (F-statistic):           4.14e-58
Time:                             14:52:03   Log-Likelihood:                -2184.9
No. Observations:                     2090   AIC:                             4394.
Df Residuals:                         2078   BIC:                             4461.
Df Model:                               11                                         
Covariance Type:                 nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                             3.9078      0.056     69.792      0.000       3.798       4.018
householdIncome                                   0.0662      0.016      4.264      0.000       0.036       0.097
householdMinor_num                               -0.0728      0.015     -4.726      0.000      -0.103      -0.043
isLivingTogether_Yes                              0.4595      0.043     10.729      0.000       0.376       0.544
metAs_workNeighbors_yes                           0.3042      0.130      2.348      0.019       0.050       0.558
metIn_school_yes                                  0.1611      0.049      3.262      0.001       0.064       0.258
partnerRace_Asian or Pacific Islander             0.1827      0.081      2.259      0.024       0.024       0.341
partnerRace_White                                 0.2158      0.043      4.963      0.000       0.131       0.301
sexFrequency_3 to 6 times a week                  0.2075      0.052      3.992      0.000       0.106       0.309
sexFrequency_Once a month or less                -0.2096      0.040     -5.283      0.000      -0.287      -0.132
sexFrequency_Once or twice a week                 0.1001      0.042      2.392      0.017       0.018       0.182
whoEarnedMore_We earned about the same amount     0.1034      0.046      2.244      0.025       0.013       0.194
==============================================================================
Omnibus:                      590.161   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1625.955
Skew:                          -1.476   Prob(JB):                         0.00
Kurtosis:                       6.156   Cond. No.                         14.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### L1 - Lasso Regression

#### GridSearchCV for Lasso

In [34]:
# CONDUCT GRID SEARCH ---------------------------
from sklearn.model_selection import GridSearchCV

# dictionary of parameters to search
param_grid = {"alpha": [0, 1, 2],
             "max_iter":[1, 20, 50, 60]}


# import lasso
from sklearn.linear_model import Lasso

lasso = Lasso()

# grid search on all values of k in dictionary
lasso_grid = GridSearchCV(lasso, param_grid, cv=5)

lasso_grid.fit(X_train, y_train)


# PRINT RESULTS ---------------------------

# best performing params (on training set)
print("best params:\n\n", lasso_grid.best_params_)

# best score parameters 
print("\nbest score:\n\n", lasso_grid.best_score_)

best params:

 {'alpha': 0, 'max_iter': 1}

best score:

 0.06462796553708876


**NOTE:** Lasso with it's best score with an alpha of 0, is just regular linear regression without regularization. For the final test, I will use Lasso with default parameters.

### L2 - Ridge Regression

#### GridSearchCV for Ridge

In [38]:
# CONDUCT GRID SEARCH ---------------------------
from sklearn.model_selection import GridSearchCV

# dictionary of parameters to search
param_grid = {"alpha": [110, 120, 130]}


# Import Ridge
from sklearn.linear_model import Ridge

ridge = Ridge()

# grid search on all values of k in dictionary
ridge_grid = GridSearchCV(ridge, param_grid, cv=5)

ridge_grid.fit(X_train, y_train)


# PRINT RESULTS ---------------------------

# best performing params (on training set)
print("best params:\n\n", ridge_grid.best_params_)

# best R-squared  performing parameters
print("\nbest score:\n\n", ridge_grid.best_score_)

best params:

 {'alpha': 120}

best score:

 0.09373438430752132


---
---

# Step 4: Compare Classification Models

### Full Model - Linear Regression (109 Features)

In [41]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train, y_train)

print("---")
print("Training set R^2: {:.2f}".format(lr.score(X_train, y_train)))
print("Test set R^2: {:.2f}".format(lr.score(X_test, y_test)))

---
Training set R^2: 0.19
Test set R^2: -63058860234115137929216.00


### Backwards Eliminated Model - Linear Regression (11 Features)

In [44]:
from sklearn.linear_model import LinearRegression

features = [    
    "householdIncome",
    "householdMinor_num",
    "isLivingTogether_Yes",
    "metAs_workNeighbors_yes",
    "metIn_school_yes",
    "partnerRace_Asian or Pacific Islander",
    "partnerRace_White",
    "sexFrequency_3 to 6 times a week",
    "sexFrequency_Once a month or less",
    "sexFrequency_Once or twice a week",
    "whoEarnedMore_We earned about the same amount"]

X = X_train[features]


lr = LinearRegression().fit(X, y_train)


print("Training set R^2: {:.2f}".format(lr.score(X, y_train)))
print("Test set R^2: {:.2f}".format(lr.score(X_test[features], y_test)))

Training set R^2: 0.13
Test set R^2: 0.09


### Ridge Regression with alpha = 120

In [45]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha = 120).fit(X_train, y_train)

print("Training set score: {:.2f}".format(ridge.score(X_train, y_train)))
print("Test set score: {:.2f}".format(ridge.score(X_test, y_test)))

Training set score: 0.17
Test set score: 0.13


### Default Lasso

In [48]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(X_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(X_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test, y_test)))
print("Number of features used:", np.sum(lasso.coef_ != 0))

Training set score: 0.00
Test set score: -0.00
Number of features used: 0


---
---

# Step 5: Conclusions